In [1]:
import pandas as pd

# Read Dataset

### Since one of datasets has several sheets which all need to be used, I need to read each sheet from one dataset first and then to read all of the dataset.

### read 3 different datasets

In [2]:
growth = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-industry-employment-growth.xlsx",sheet_name='Growth from Industry Transition')
skill_pen = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-skill-penetration.xlsx",sheet_name='Skill Penetration')
ind_skill = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-industry-skills-needs.xlsx",sheet_name='Industry Skills Needs')

### read 3 sheets in one dataset 

In [3]:
industry=pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Industry Migration')
skill = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Skill Migration')
country = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Country Migration')

### read each sheet in one dataset

In [4]:
education_major= pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/education_and_major.xlsm",sheet_name="Major")
education_skill= pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/education_and_major.xlsm",sheet_name="Skill")

# Select Certain Columns 

### Since we need to combine all datasets together, we need to choose the primary key and foreign key of each dataset first. Then we pick up the columns which we need. All columns we need are shown in the coding part.

In [5]:
growth_from_ind=growth[['country_name','wb_income','wb_region','isic_section_name','industry_name','growth_rate_2015','growth_rate_2016','growth_rate_2017','growth_rate_2018']]
industry_migration=industry[['country_name','wb_income','wb_region','isic_section_name','industry_name','net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018']]
skill_migration=skill[['country_name','wb_income','wb_region','skill_group_category','skill_group_name','net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018']]

In [6]:
skill_penetration=skill_pen[['year','isic_section_name','industry_name','skill_group_category','skill_group_name','skill_group_penetration_rate']]
ind_skill_needs=ind_skill[['year','isic_section_name','industry_name','skill_group_category','skill_group_name','skill_group_rank']]

### Check Each Column's Data Type of Each Dataset

In [7]:
growth_from_ind.dtypes
skill_penetration.dtypes
ind_skill_needs.dtypes
industry_migration.dtypes
skill_migration.dtypes

country_name             object
wb_income                object
wb_region                object
skill_group_category     object
skill_group_name         object
net_per_10K_2015        float64
net_per_10K_2016        float64
net_per_10K_2017        float64
net_per_10K_2018        float64
dtype: object

### Delete all the null value

In [8]:
skill_penetration=skill_penetration[pd.notnull(skill_penetration['skill_group_penetration_rate'])]
ind_skill_needs=ind_skill_needs[pd.notnull(ind_skill_needs['skill_group_rank'])]

In [9]:
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2015'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2016'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2017'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2018'])]

# Combine Various Dataset

### Step 1 combine skill penetration and industry skill needs called skill_part by using the outer merge. Although it will have some NA values, I still want to keep all the industry name and skill name.

In [10]:
skill_part=pd.merge(skill_penetration,ind_skill_needs, how='outer', on=['year','isic_section_name','industry_name','skill_group_category','skill_group_name'])

### Step 2 seperate the skill group penetration rate according to the year.Since the origianl file is vertical, I want to change them to horizontal for the further combination.

In [11]:
df1 = skill_part[skill_part['year'] == 2015].drop(columns=['year'])
df2 = skill_part[skill_part['year'] == 2016].drop(columns=['year'])
df3 = skill_part[skill_part['year'] == 2017].drop(columns=['year'])
df4 = skill_part[skill_part['year'] == 2018].drop(columns=['year'])

### Step 3 rename the seperated skill group penetration rate

In [12]:
df1=df1.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2015','skill_group_rank':'skill_group_rank_2015'})
df2=df2.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2016','skill_group_rank':'skill_group_rank_2016'})
df3=df3.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2017','skill_group_rank':'skill_group_rank_2017'})
df4=df4.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2018','skill_group_rank':'skill_group_rank_2018'})

### Step 4 combine the the skill group penetration rate as a new table called final_skill_part.Here I will use the outer to combine all the skill group penetration rate. 

In [13]:
final_skill_part=pd.merge(df1,df2, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])
final_skill_part=pd.merge(final_skill_part,df3, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])
final_skill_part=pd.merge(final_skill_part,df4, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])

In [14]:
final_skill_part=final_skill_part[['isic_section_name','industry_name','skill_group_category','skill_group_name',"skill_group_penetration_rate_2015",'skill_group_penetration_rate_2016',"skill_group_penetration_rate_2017","skill_group_penetration_rate_2018","skill_group_rank_2015","skill_group_rank_2016","skill_group_rank_2017","skill_group_rank_2018"]]

### Step 5 combine industry_migration with final_skill_part and drop the net_per_10k_YYYY columns called industry_combination. I will use the left combine to merge to datasetst since I want to keep all the industry index and name information.

In [15]:
industry_combination=pd.merge(industry_migration,final_skill_part,how='left',on=['isic_section_name','industry_name'])
industry_combination=industry_combination.drop(columns=['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018'])

### Step 6 combine the skill_migration with fill_skill part called skill_combination and drop the duplicated skill group penetration and skill group rank. I use the left merge to keep all the skill information such as name and category.

In [16]:
skill_combination=pd.merge(skill_migration,final_skill_part,how='left',on=['skill_group_category','skill_group_name'])
skill_combination=skill_combination.drop(columns=['skill_group_penetration_rate_2015','skill_group_penetration_rate_2016','skill_group_penetration_rate_2017','skill_group_penetration_rate_2018'])
skill_combination=skill_combination.drop(columns=['skill_group_rank_2015','skill_group_rank_2016','skill_group_rank_2017','skill_group_rank_2018'])

### Step 7 Combine industry combination and skill combination dataset called ind_skill_combination.I use the inner combination to deal with the large amount of NA values.It still show all the information that I want to keep about the industry and skill.

In [17]:
ind_skill_combination=pd.merge(skill_combination,industry_combination,how='inner',on=['country_name','wb_income','wb_region','isic_section_name','industry_name','skill_group_category','skill_group_name'])

### Step 8 Combine education skill and education major to be a new education table called education. The inner combination help me to deal with the NA value of the education. Actually, it does not have any NA values to be deleted during the inner combination.

In [18]:
education=pd.merge(education_skill,education_major,how='inner',on=['Program'])

### Step 9 Final Combine the ind_skill_combination and education called final_dataset. The inner combination could help me to delete the useless NA values but keep all the informations that I need

In [19]:
final_dataset=pd.merge(ind_skill_combination,education,how='inner',on=['skill_group_name'])

# The output of the final dataset

In [20]:
final_dataset.to_csv(r'/Users/tanya/Desktop/IS590 OM/OM DATASETS/skill_migration_features.csv')